<a href="https://colab.research.google.com/github/AtaruOhto/google-colab-sandbox/blob/master/xgboost_sandbox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

train_data = pd.read_csv("./train.csv")
y = train_data.y.values
train_data = train_data.drop(["id", "y"], axis=1)
train_data["default"] = train_data["default"].map(dict(no=0, yes=1)).astype('int64')
train_data["housing"] = train_data["housing"].map(dict(no=0, yes=1)).astype('int64')
train_data["loan"] = train_data["loan"].map(dict(no=0, yes=1)).astype('int64')
train_data["month"] = train_data["month"].map(dict(jan=1, feb=2, mar=3, apr=4, may=5, jun=6, jul=7, aug=8, sep=9, oct=10, nov=11, dec=12)).astype('int64')
X = pd.get_dummies(train_data ,columns=['job', "marital", "education", "contact", "poutcome"])

train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.2, shuffle=True) 
train_x

,age,default,balance,housing,loan,day,month,duration,campaign,pdays,previous,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,marital_divorced,marital_married,marital_single,education_primary,education_secondary,education_tertiary,education_unknown,contact_cellular,contact_telephone,contact_unknown,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
20568,32,0,5041,1,0,7,11,101,1,237,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0
3053,33,0,15380,1,0,27,5,345,1,299,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1
8931,34,0,11160,1,0,30,4,82,1,206,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,1
4242,31,0,76013,1,0,27,5,345,1,353,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1
26481,30,0,76002,0,0,18,7,158,3,670,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2086,30,0,25951,1,1,4,8,158,4,479,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1
25302,29,0,29068,1,0,17,5,158,1,137,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,1
18637,32,0,62305,0,0,2,6,117,1,759,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1
18919,29,0,31865,0,0,18,5,158,3,340,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1


In [68]:
# dtrain = xgb.DMatrix(train_x, label=train_y)  

# param = {'max_depth': 4, 
#     'eta': 0.3, 
#     'objective': 'binary:logistic',
#     'eval_metric': 'auc'
# }

# evallist = [(dvalid, 'eval'), (dtrain, 'train')]  

# num_round = 6
# bst = xgb.train(param, dtrain, num_round)
# dtest = xgb.DMatrix(test_x)  
# pred = bst.predict(dtest) 

# predictions = [round(value) for value in pred]

# from sklearn.metrics import accuracy_score  

# test_y
# predictions

# score = accuracy_score(test_y, predictions)  
# score
# # print('score:{0:.4f}'.format(score))  

# # xgb.plot_importance(bst) 
# # xgb.to_graphviz(bst)

0.9265682656826568

In [72]:
dtrain = xgb.DMatrix(train_x, label=train_y)
dvalid = xgb.DMatrix(test_x, label=test_y)

param = {'max_depth': 4, 
    'eta': 0.3, 
    'objective': 'binary:logistic',
    'eval_metric': 'auc'
}

evallist = [(dvalid, 'eval'), (dtrain, 'train')]  
num_round = 10000  
bst = xgb.train(param, dtrain, num_round, evallist, early_stopping_rounds=5) 

[0]	eval-auc:0.774466	train-auc:0.771234
Multiple eval metrics have been passed: 'train-auc' will be used for early stopping.

Will train until train-auc hasn't improved in 5 rounds.
[1]	eval-auc:0.789499	train-auc:0.789456
[2]	eval-auc:0.791767	train-auc:0.790901
[3]	eval-auc:0.789233	train-auc:0.789041
[4]	eval-auc:0.804054	train-auc:0.802178
[5]	eval-auc:0.808833	train-auc:0.809161
[6]	eval-auc:0.81664	train-auc:0.816635
[7]	eval-auc:0.818876	train-auc:0.820676
[8]	eval-auc:0.823757	train-auc:0.825962
[9]	eval-auc:0.830374	train-auc:0.833141
[10]	eval-auc:0.831799	train-auc:0.836832
[11]	eval-auc:0.836531	train-auc:0.84356
[12]	eval-auc:0.836888	train-auc:0.846735
[13]	eval-auc:0.83733	train-auc:0.849172
[14]	eval-auc:0.837872	train-auc:0.851082
[15]	eval-auc:0.840079	train-auc:0.853781
[16]	eval-auc:0.842944	train-auc:0.857132
[17]	eval-auc:0.843574	train-auc:0.859515
[18]	eval-auc:0.84444	train-auc:0.861473
[19]	eval-auc:0.846861	train-auc:0.865226
[20]	eval-auc:0.846116	train-auc

In [73]:
print('Best Score:{0:.4f}, Iteratin:{1:d}, Ntree_Limit:{2:d}'.format(  
     bst.best_score, bst.best_iteration, bst.best_ntree_limit)) 

Best Score:0.9997, Iteratin:868, Ntree_Limit:869
